In [4]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [5]:
spark = SparkSession.builder.master("local").appName("examples").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/14 08:09:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = spark.createDataFrame(
    [
        (8, "Liam", 4),
        (9, "Colin", 15),
    ]
).toDF("id", "first_name", "age")

In [9]:
def with_life_stage(col_name):
    def _(df):
        col = F.col(col_name)
        return df.withColumn(
            "life_stage",
            F.when(col < 13, "child")
            .when(col.between(13, 19), "teenager")
            .when(col > 19, "adult"),
        )

    return _

In [11]:
df.transform(with_life_stage("age")).show()

+---+----------+---+----------+
| id|first_name|age|life_stage|
+---+----------+---+----------+
|  8|      Liam|  4|     child|
|  9|     Colin| 15|  teenager|
+---+----------+---+----------+



## This syntax only works in PySpark 3.3+

In [7]:
def with_life_stage2(df, col_name):
    col = F.col(col_name)
    return df.withColumn(
        "life_stage",
        F.when(col < 13, "child")
        .when(col.between(13, 19), "teenager")
        .when(col > 19, "adult"),
    )

In [12]:
df.transform(with_life_stage2, "age").show()

TypeError: transform() takes 2 positional arguments but 3 were given

In [ ]:
newAddressesToInsert = (
    updatesDF.alias("updates")
    .join(customersTable.toDF().alias("customers"), "customerid")
    .where("customers.current = true AND updates.address <> customers.address")
)